Dans ce dernier TP d'apprentissage, nous allons expérimenter du traitement du langage naturel à travers deux tâches classiques du domaines : la **classification de texte** et la **génération de texte**.

# Présentation des données

Commencez par télécharger les données :

In [ ]:
!wget http://acarlier.fr/tp/surname-nationality.csv

Cette base de données regroupe des noms de famille usuels (les plus communs) dans 37 pays du monde. Le fichier CSV contient plus de 36000 noms (*surnames*) accompagnés de la nationalité associée. Bien évidemment, un nom peut apparaître dans plusieurs pays du monde (par exemple, le nom Lopez est d'après cette base très usuel au Honduras, au Chili, au Brésil, au Venezuela, au Nicaragua, au Pérou, et en Espagne !).

In [ ]:
import csv
import numpy as np

# Initialisation de listes pour ranger les noms et nationalités
surnames = []
nationalities = []

# Lecture du fichier CSV
with open('surname-nationality.csv', 'r', newline='') as file:
    reader = csv.reader(file)
    next(reader)  # Header
    for row in reader:
        surnames.append(row[1].lower())
        nationalities.append(row[2])

print(surnames)
print(nationalities)

# Classification de texte

Dans un premier temps, nous allons tenter de résoudre un problème de classification de texte : est-il possible de prédire la nationalité d'une personne uniquement à partir de son nom ?

Pour les raisons évoquées précédemment, certaines nationalités risquent d'être indissociables car de nombreux noms de familles usuels sont communs dans les mêmes pays. Ainsi, pour simplifier le problème, nous allons nous circonscrire à une dizaine de pays dont les noms sont de consonance suffisamment diverse :

In [ ]:
nationality_classes = ['Spanish', 'Korean', 'Portuguese', 'French', 'Arabic', 'Indian', 'Italian', 'Vietnamese', 'Irish', 'German']

Il nous faut donc préparer les données pour ce problème.

Pour commencer, on extrait les listes de noms/nationalités correspondant aux nationalités listées ci-dessus, puis on associe un id unique à chaque nationalité : ce sera la classe à prédire.

In [ ]:
# Création d'un dictionnaire pour associer chaque nationalité à un indice
nationality_to_label = {nationality: idx for idx, nationality in enumerate(nationality_classes)}

selected_surnames = []
selected_nationalities = []
# Sélection du sous-ensemble de couples noms/nationalités pour les pays sélectionnés
for s, n in zip(surnames, nationalities):
  if n in nationality_classes:
    selected_surnames.append(s)
    selected_nationalities.append(n)

# Génération des labels associés à ces nationalités
labels = [nationality_to_label[nationality] for nationality in selected_nationalities]

# Conversion de la liste en tableau numpy
Y = np.array(labels)

print("Labels (Y):")
print(Y)
print(Y.shape)
print("\nDictionnaire Nationalité -> Indice")
print(nationality_to_label)

Il nous faut maintenant préparer les données $X$. Cette partie est plus compliquée car elle nécessite deux étapes importantes :     
1.   Convertir les caractères textuels en données numériques
2.   Gérer le problème des séquences de longueur variable

Pour la conversion en données numériques, il faut d'abord décider de l'unité de modélisation du texte : on parle de *tokens*. Ici, il semble assez naturel de considérer chaque caractère du nom comme un *token*. On va ensuite associer un indice à chaque caractère.

Le problème des séquences de longueur variable est un problème à cause des structures de données que nous utilisons. Si chaque nom est codé par une séquence d'entiers, notre ensemble de données $X$ va donc être représenté par une matrice où chaque ligne correspond à un nom et compte donc un nombre potentiellement différent d'entiers sur les colonnes. Cela n'est pas possible car les tableaux Numpy nécessitent que toutes les lignes aient le même nombre de colonnes.

On va donc adopter une solution simple qui consiste à détecter la longueur maximale d'une séquence, et à faire en sorte que toutes les séquences soient complétées par un *token* spécial (que l'on va appeler **pad**) pour atteindre la longueur maximale.

On commence donc par trouver la longueur maximale d'une séquence :

In [ ]:
MAX_LEN = 0
# Parcours des noms du dataset
for s in selected_surnames:
  # A COMPLETER

print(MAX_LEN)

On détermine ensuite le vocabulaire, c'est-à-dire l'ensemble des caractères présents dans la base de données, auxquels on va assigner un indice de 1 à *vocab\_size* (taille du vocabulaire). L'indice 0 correspondra au caractère **pad** que l'on utilisera pour compléter toutes les séquences à la longueur de la séquence maximale.

In [ ]:
# Concaténation de tous les noms du dataset
all_names = ''
for s in selected_surnames:
  all_names = all_names + s

# Détermination des caractères apparaissant au moins une fois, et tri de ces caractères
vocab = sorted(set(all_names))
# Ajout d'un token pour la complétion des séquences (padding)
vocab.insert(0, '<pad>')

# Création d'un dictionnaire pour associer chaque token à un indice
char_to_id = {char: idx for idx, char in enumerate(vocab)}

# Taille du vocabulaire final
VOCAB_SIZE = len(vocab)

print(vocab)
print(char_to_id)

Il reste donc à générer la séquence associée à chaque nom. Pour cela on doit prendre en compte le fait que la longueur maximale d'une séquence est de 18, et que chaque caractère est codé par un entier :     

|  \<pad\>  |       |   '   |  ,    |  a    |  b    |  c    |  ...  |
|---    |:-:    |:-:    |:-:    |--:    |--:    |--:    |--:    |
|  0    |    1  |   2   |   3   |   4   |   5   |   6   |  ...  |

Ainsi, le nom "o'sullivan" est réécrit comme la séquence de *tokens* suivante :

``` <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> o ' s u l l i v a n ```  

qui va donc être codée ainsi :
$[0, 0, 0, 0, 0, 0, 0, 0, 18, 2, 22, 24, 15, 15, 12, 25, 4, 17]$

In [ ]:
X = np.zeros((Y.shape[0], MAX_LEN))

for idx, s in enumerate(selected_surnames):
  s_id = [0] * (MAX_LEN - len(s)) + [char_to_id[char] for char in s] # On ajoute le bon nombre de 0 au début de la séquence
  X[idx, :] = np.array(s_id)

print(X)

On dispose donc maintenant de nos données $X$ et de nos étiquettes $Y$, il reste à les séparer en deux ensembles d'apprentissage et de test :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

Le problème de classification de texte nécessite d'associer un label à une séquence de caractères : c'est donc un problème *Many-to-one*.


<center><img src="https://drive.google.com/uc?id=1dkccEF--TaRvLMr0zFFQJQQdFYu7wMzF" width=600> </center>
<caption><center> Schéma d'une classification de nom à l'aide d'une cellule récurrente</center></caption>

Nous allons construire le réseau le plus simple possible pour résoudre ce problème. Ce réseau sera composé de 3 couches :

<center><img src="https://drive.google.com/uc?id=1FiYjHu-YZS9bYZv5GBxSIGPsC-nEJJvA" width=600> </center>
<caption><center> Réseau à construire</center></caption>


1.   Une couche d'[Embedding](https://keras.io/api/layers/core_layers/embedding/) pour transformer chaque *token* en un vecteur de dimension *embedding\_size*.

<center><img src="https://drive.google.com/uc?id=13OG0O6-_7BdOKnvYQCsQM5PxZKcmAe95" width=300> </center>

2.   Une couche [récurrente](https://keras.io/api/layers/recurrent_layers/simple_rnn/) simple comportant un nombre de neurones à définir.

3.   Une couche de sortie (Dense) qui va réaliser la classification.


Vous pouvez utiliser la même valeur pour *embedding\_size* et le nombre de neurones de la couche récurrente, par exemple 128.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models

# A COMPLETER
model = models.Sequential()
model.add(...)
model.add(...)
model.add(...)
model.summary()

Attention, comme on l'a vu dans le TP5, il faut formater les labels en *one-hot vectors*

In [ ]:
Y_train_cat = keras.utils.to_categorical(Y_train, num_classes=10)
Y_test_cat = keras.utils.to_categorical(Y_test, num_classes=10)


In [ ]:
# A COMPLETER
model.compile(loss=...,
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train_cat, validation_data=(X_test, Y_test_cat), epochs=30, batch_size=32)

Bien que le modèle apprenne toujours après 30 epochs, on observe un début de sur-apprentissage et il est donc raisonnable de s'arrêter là. Les résultats sont plutôt bons, jusqu'à 68% de bonnes classifications sur l'ensemble de test. Evidemment le réseau est encore probablement trop simple pour obtenir de meilleurs résultats.

## Version avec LSTM

On a vu en cours que les cellules récurrentes avaient des difficultés à apprendre les dépendances à long terme, et que l'on pouvait utiliser des LSTM pour pallier à ce problème. Essayez de remplacer la cellule RNN par un LSTM et voyez la différence :

On observe en effet que le modèle généralise un peu mieux !

# Génération de texte

Passons maintenant à la génération de texte : on va créer un modèle de langage qui pourra ensuite nous aider à générer de nouveaux noms. Dans cette partie, on ne s'intéresse plus à la nationalité.

Le problème est ici différent, de la classe *Many-to-Many*.


<center><img src="https://drive.google.com/uc?id=1TfgprY0yB4blIlRbHHr3S8UQB3nid5zo" width=600> </center>
<caption><center> Génération de nom à l'aide d'une cellule récurrente</center></caption>

Le formatage des données est similaire à la partie précédente mais on va ajouter deux nouveaux *tokens* : l'un pour marquer le début d'un nom (\<sos\>, *start of surname*) et l'autre pour en marquer la fin (\<eos\>, *end of surname*).




Cette fois-ci nous allons utiliser tous les noms de la base initiale, il faut donc recalculer la longueur maximale d'une séquence :

In [ ]:
MAX_LEN = 0
# Parcours des noms du dataset
for s in surnames:
  if len(s) > MAX_LEN:
    MAX_LEN = len(s)

print(MAX_LEN)

On regénère également un vocabulaire (il y a un nouveau caractère, le '-', qui apparait dans certaines langues) en y adjoignant les deux nouveaux tokens.

|  \<pad\>  |       |   '   |  ,    |  -   |  a    |  b    |  c    |  ...  |   z    |   \<sos\>    |   \<eos\>   |
|---    |:-:    |:-:    |:-:    |:-:    |:--:    |:--:    |:--:    |:--:    |:--:    |:--:    |:--:    |
|  0    |    1  |   2   |   3   |   4   |   5   |   6   |   7   |  ...  |   30    |  31   |   32  |  


In [ ]:
# Concaténation de tous les noms du dataset
all_names = ''
for s in surnames:
  all_names = all_names + s

# Détermination des caractères apparaissant au moins une fois, et tri de ces caractères
vocab = sorted(set(all_names))
# Ajout d'un token pour la complétion des séquences (padding), ainsi que pour
# le début et la fin des noms
vocab.insert(0, '<pad>')
vocab.append('<sos>') # Start of surname
vocab.append('<eos>') # End of surname

# Création d'un dictionnaire pour associer chaque token à un indice
char_to_id = {char: idx for idx, char in enumerate(vocab)}
# Taille du vocabulaire final
VOCAB_SIZE = len(vocab)

print(vocab)
print(char_to_id)

Autre différence avec la classification de texte, ici les labels $Y$ sont également des séquences ! Pour la génération de texte, nous allons ajouter les *tokens* de padding à la fin et non au début.


Ainsi, le nom "o'sullivan" est réécrit comme la séquence de *tokens* suivante :

``` <sos> o ' s u l l i v a n <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> ```  

qui va donc être codée ainsi :
$[31, 19, 2, 23, 25, 16, 16, 13, 26, 5, 18, 32, 0, 0, 0, 0, 0, 0, 0, 0]$

Cette séquence est donc de taille MAX_LEN + 2 (on a ajouté deux *tokens* au début et à la fin du nom).

La donnée $X$ associée à cette séquence est donc le vecteur $[31, 19, 2, 23, 25, 16, 16, 13, 26, 5, 18, 32, 0, 0, 0, 0, 0, 0, 0]$ des MAX_LEN + 1 premiers caractères, et le label $Y$ à prédire est le vecteur $[19, 2, 23, 25, 16, 16, 13, 26, 5, 18, 32, 0, 0, 0, 0, 0, 0, 0, 0]$ des MAX_LEN + 1 derniers caractères.

In [ ]:
# Préparation des structures de donnée X et Y
X = np.zeros((len(surnames), MAX_LEN+1))
Y = np.zeros((len(surnames), MAX_LEN+1))

for idx, s in enumerate(surnames):
  # A COMPLETER


print(X)
print(Y)

A nouveau, il ne faut pas oublier de transformer les labels en *one-hot vectors* :

In [ ]:
# A COMPLETER
Y_cat = ...

Nous allons construire un réseau similaire à celui de la classification, excepté que cette fois il y aura une sortie pour chaque élément de la séquence d'entrée :   

<center><img src="https://drive.google.com/uc?id=1KDA28fEeLwM5_bZUOifJNK-b3YdDF7WT" width=600> </center>
<caption><center> Réseau à construire</center></caption>

Pour cela, lorsque vous mettre en place la couche récurrente (utilisez directement un LSTM, qui fonctionne mieux), positionnez correctement l'attribut ```return_sequences```.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models

# A COMPLETER
model = models.Sequential()
...
model.summary()

L'apprentissage est un peu plus long (même pour seulement 10 epochs), donc n'hésitez pas à prendre le temps de lire le bloc de code suivant (pour générer des noms) pour ne pas perdre de temps.

In [ ]:
# A COMPLETER
model.compile(loss=...,
              optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
              metrics=['accuracy'])

history = model.fit(X, Y_cat, epochs=10, batch_size=32)

Une fois le modèle entraîné, nous pouvons maintenant générer un nouveau nom, soit de zéro ou soit en partant d'un début de séquence *seed* (**le code est fourni, prenez le temps de le comprendre !**).

Notez bien que l'on utilise ce modèle de langage de manière stochastique. Plutôt que de choisir systématiquement le caractère le plus probable, on tire aléatoirement un caractère en suivant la distribution de probabilité prédite par le réseau, ce qui permet de générer plusieurs fins de nom possibles pour un même début de séquence ! (A vous de tester un peu)

In [ ]:
# Début de séquence : mettre '' si l'on veut générer un nom de zéro
seed_seq = 'macro'

# Création de la séquence qui va être fournie en entrée du réseau :
# On ajoute un token <sos> au démarrage, et on transcrit en la séquence d'id correspondante
input_seq = [char_to_id['<sos>']]
for s in seed_seq:
  input_seq.append(char_to_id[s])

last_char = -1
i = 0

# On génère des séquences de taille inférieure à MAX_LEN, et on s'arrête lorsque
# l'on génère un token <eos> (id 32)
while i <= MAX_LEN and last_char != 32:
  # La séquence d'entrée doit être de dimension BATCH_SIZE x SEQ_LEN x 1
  # soit en fait ici 1 x SEQ_LEN x 1
  input = np.array(input_seq)
  input = np.expand_dims(input, 0)
  input = np.expand_dims(input, 2)

  # Prédiction du modèle sur la séquence en cours
  pred = model.predict(input, verbose=0)

  # Échantillonnage du caractère généré à partir de la distribution de probabilité
  # prédite par le modèle pour le dernier élément de la séquence
  last_char = np.random.choice(33, 1, p=pred[0, -1])[0]

  # Ajout du caractère à la séquence générée
  input_seq.append(last_char)
  i += 1

# Affichage du nom généré
generated_surname = ''
for s in input_seq:
  if s != 31 and s != 32:
    generated_surname+=vocab[s]

print(generated_surname)